## Using a simple RNN for machine translation
This project is still in progress.

I'm busy doing the Udacity Natural Language Processing Nanodegree and I'm busy trying build an effective English to French translation model.

<a href='http://www.manythings.org/anki/'>Data</a>

In [1]:
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras import Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM, Flatten
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


In [2]:
fname = 'nld-eng/nld.txt'
df = pd.read_csv(fname, header=None, sep='\t', lineterminator='\n')
df[:10]

,0,1
0,Hi.,Hoi.
1,Run!,Ren!
2,Who?,Wie?
3,Wow!,Da's niet gek!
4,Fire!,Vuur!
5,Fire!,Brand!
6,Help!,Help!
7,Jump.,Spring.
8,Stop!,Stop!
9,Stop!,Halt!


In [3]:
english = list(df.iloc[:, 0])
dutch = list(df.iloc[:, 1])

In [4]:
# Preprosessing  functions
def tokenize(x):
    text_tokenizer = Tokenizer()
    text_tokenizer.fit_on_texts(x)
    text_tokenized = text_tokenizer.texts_to_sequences(x)
    return text_tokenized, text_tokenizer

def pad(x, length=None):
    if length == None:
        length = max([len(sentance) for sentance in x])
    x = np.array(x)
    x_padded = pad_sequences(x, maxlen=length, dtype='int32', padding='post')
    return x_padded

def preprocess(x, y):
    preprocess_x, x_token = tokenize(x)
    preprocess_y, y_token = tokenize(y)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    return preprocess_x, preprocess_y, x_token, y_token

In [5]:
eng_tokenized, eng_tokenizer = tokenize(english)
dut_tokenized, dut_tokenizer = tokenize(dutch)

In [6]:
eng_padded = pad(eng_tokenized, 10)
dut_padded = pad(dut_tokenized, 10)

In [13]:
# print(LSTM.__doc__)
eng_padded.shape

(28224, 10)

In [18]:
unique_eng_words = len(eng_tokenizer.word_index)
unique_dut_words = len(dut_tokenizer.word_index)

model = Sequential()
model.add(Embedding(unique_eng_words, 50, input_length=10))
model.add(LSTM(10, input_shape=(50, 10), return_sequences=True))
model.add(TimeDistributed(Dense(unique_dut_words)))

model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 10, 50)            349400    
_________________________________________________________________
lstm_9 (LSTM)                (None, 10, 10)            2440      
_________________________________________________________________
time_distributed_6 (TimeDist (None, 10, 9585)          105435    
Total params: 457,275
Trainable params: 457,275
Non-trainable params: 0
_________________________________________________________________


In [20]:
dut_padded.reshape(-1, 10, 1)

(28224, 10)

In [29]:
dut_reshaped = dut_padded.reshape(-1, 10, 1)
model.fit(eng_padded, dut_reshaped, batch_size = 1000)

Epoch 1/1
28224/28224 [==============================] - 51s 2ms/step - loss: 9.9745

In [34]:
prediction = model.predict(eng_padded[:1])
prediction.shape

(1, 10, 9585)

In [58]:
prediction = np.argmax(prediction, axis=1)
num2word = dict((num, word) for word, num in eng_tokenizer.word_index.items())

In [56]:
for i, num in enumerate(prediction[0]):
    if i == 10: break
    print(num2word[num])

the
in
in
in
you
in
in
in
in
in
